**Mounting Google Drive**

In [1]:
from google.colab import drive 
drive.mount('/content/drive')
%cd/content/drive

Mounted at /content/drive
/content/drive


**Importing Libraries**

In [13]:
!pip install keras-metrics
!pip install -q -U keras-tuner



In [15]:
#1-1: Import installed libraries
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from sklearn import preprocessing, ensemble  
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, make_scorer, precision_score, f1_score

# Use scikit-learn to grid search the learning rate and momentum
import tensorflow as tf
import keras
import keras_metrics
import kerastuner as kt
from keras import layers
from keras.layers import Dense
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
from kerastuner.tuners import RandomSearch


**Reading dataset**

In [4]:
# 2-1: Read csv as dataframe
custchurn=pd.read_csv("/content/drive/My Drive/ytd_churn_data.csv") 
pd.set_option('display.max_column',None)

**Data Preparation**

In [6]:
#4-1: Change 'SeniorCitizen' from interger to categorical variable
custchurn['SeniorCitizen']=pd.Categorical(custchurn['SeniorCitizen'])
print(custchurn.dtypes)

customerID            object
gender                object
SeniorCitizen       category
Partner               object
Dependents            object
tenure                 int64
PhoneService          object
MultipleLines         object
InternetService       object
OnlineSecurity        object
OnlineBackup          object
DeviceProtection      object
TechSupport           object
StreamingTV           object
StreamingMovies       object
Contract              object
PaperlessBilling      object
PaymentMethod         object
MonthlyCharges       float64
TotalCharges         float64
Churn                 object
dtype: object


In [7]:
#4-2: Replace target variable 'Churn' from 'Yes=1, 'No'=0, and drop 'customerID'

custchurn['Churn'].replace(to_replace='Yes', value=1, inplace=True)
custchurn['Churn'].replace(to_replace='No',  value=0, inplace=True)
custchurn=custchurn.drop(['customerID'], axis=1)

#4-2: Change 16 categorical to dummy variables
custchurn_dummy = pd.get_dummies(custchurn)
print(f"Dataset Total records, Total variables: {custchurn_dummy.shape}")
custchurn_dummy.head()

#4-3: Normalise 3 numerical variables
#from sklearn.preprocessing import StandardScaler
#num_cols = ["tenure", 'MonthlyCharges', 'TotalCharges']
#scaler= StandardScaler()
#custchurn_dummy[num_cols] = scaler.fit_transform(custchurn_dummy[num_cols])
#custchurn_dummy.head(20)


Dataset Total records, Total variables: (7043, 47)


,tenure,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male,SeniorCitizen_0,SeniorCitizen_1,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,PhoneService_Yes,MultipleLines_No,MultipleLines_No phone service,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,1,29.85,29.85,0,1,0,1,0,0,1,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0
1,34,56.95,1889.50,0,0,1,1,0,1,0,1,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,1
2,2,53.85,108.15,1,0,1,1,0,1,0,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1
3,45,42.30,1840.75,0,0,1,1,0,1,0,1,0,1,0,0,1,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,1,0,1,0,1,0,0,0
4,2,70.70,151.65,1,1,0,1,0,1,0,1,0,0,1,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0


In [8]:
print(custchurn_dummy.dtypes)

tenure                                       int64
MonthlyCharges                             float64
TotalCharges                               float64
Churn                                        int64
gender_Female                                uint8
gender_Male                                  uint8
SeniorCitizen_0                              uint8
SeniorCitizen_1                              uint8
Partner_No                                   uint8
Partner_Yes                                  uint8
Dependents_No                                uint8
Dependents_Yes                               uint8
PhoneService_No                              uint8
PhoneService_Yes                             uint8
MultipleLines_No                             uint8
MultipleLines_No phone service               uint8
MultipleLines_Yes                            uint8
InternetService_DSL                          uint8
InternetService_Fiber optic                  uint8
InternetService_No             

In [9]:
#4-4: Split to test and train dataset 70:30 ratio, check ratio of 0 and 1 in target variable

x=custchurn_dummy.drop(['Churn'], axis=1)
y=custchurn_dummy.Churn
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=0)

print("x_train, x_test:", x_train.shape, x_test.shape)
print("y_train, y_test:", y_train.shape, y_test.shape)
print(y_train.value_counts(), '\n', y_test.value_counts())




x_train, x_test: (4930, 46) (2113, 46)
y_train, y_test: (4930,) (2113,)
0    3614
1    1316
Name: Churn, dtype: int64 
 0    1560
1     553
Name: Churn, dtype: int64


**Define model build**

In [46]:
# Hyperparameter tuning function : Hidden layers (2 to 20)
#                                : neurons (32 to 218)
#                                : learning rate (0.01, 0.001, 0.0001)

def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=218,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model


**Tuning with Method 1 : Hyperband**

In [35]:
# Instantiate the Hyperband tuner to perform the hypertuning

tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=20,
    factor=3,
    directory='My Drive',
    project_name='Colab Notebooks')


In [36]:
# Initiate stop_early

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [37]:
tuner.search(x_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])


Trial 30 Complete [00h 00m 11s]
val_accuracy: 0.8012170195579529

Best val_accuracy So Far: 0.8174442052841187
Total elapsed time: 00h 02m 55s
INFO:tensorflow:Oracle triggered exit


In [ ]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]


In [39]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, y_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
124/124 [==============================] - 2s 7ms/step - loss: 0.7847 - accuracy: 0.6917 - val_loss: 0.5153 - val_accuracy: 0.7769
Epoch 2/50
124/124 [==============================] - 1s 5ms/step - loss: 0.5577 - accuracy: 0.7512 - val_loss: 0.5668 - val_accuracy: 0.7505
Epoch 3/50
124/124 [==============================] - 1s 5ms/step - loss: 0.5674 - accuracy: 0.7412 - val_loss: 0.5143 - val_accuracy: 0.7708
Epoch 4/50
124/124 [==============================] - 1s 5ms/step - loss: 0.5522 - accuracy: 0.7569 - val_loss: 0.6861 - val_accuracy: 0.5091
Epoch 5/50
124/124 [==============================] - 1s 5ms/step - loss: 0.5570 - accuracy: 0.7387 - val_loss: 0.5209 - val_accuracy: 0.7779
Epoch 6/50
124/124 [==============================] - 1s 5ms/step - loss: 0.5336 - accuracy: 0.7640 - val_loss: 0.5132 - val_accuracy: 0.7769
Epoch 7/50
124/124 [==============================] - 1s 4ms/step - loss: 0.5351 - accuracy: 0.7644 - val_loss: 0.5022 - val_accuracy: 0.7738
Epoch 

In [40]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(x_test, y_test, epochs=best_epoch)

Epoch 1/14
67/67 [==============================] - 1s 4ms/step - loss: 0.6117 - accuracy: 0.7067
Epoch 2/14
67/67 [==============================] - 0s 4ms/step - loss: 0.5715 - accuracy: 0.7326
Epoch 3/14
67/67 [==============================] - 0s 4ms/step - loss: 0.5708 - accuracy: 0.7386
Epoch 4/14
67/67 [==============================] - 0s 4ms/step - loss: 0.5632 - accuracy: 0.7398
Epoch 5/14
67/67 [==============================] - 0s 4ms/step - loss: 0.5743 - accuracy: 0.7401
Epoch 6/14
67/67 [==============================] - 0s 4ms/step - loss: 0.5418 - accuracy: 0.7480
Epoch 7/14
67/67 [==============================] - 0s 4ms/step - loss: 0.5162 - accuracy: 0.7510
Epoch 8/14
67/67 [==============================] - 0s 5ms/step - loss: 0.5120 - accuracy: 0.7762
Epoch 9/14
67/67 [==============================] - 0s 5ms/step - loss: 0.5044 - accuracy: 0.7551
Epoch 10/14
67/67 [==============================] - 0s 4ms/step - loss: 0.5066 - accuracy: 0.7586
Epoch 11/14
67/67 [

In [41]:
eval_result = hypermodel.evaluate(x_test, y_test)
print("[test loss, test accuracy]:", eval_result)

67/67 [==============================] - 0s 1ms/step - loss: 0.5160 - accuracy: 0.7383
[test loss, test accuracy]: [0.5159661769866943, 0.7382867932319641]


In [42]:
hypermodel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                3008      
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 96)                3168      
_________________________________________________________________
dense_6 (Dense)              (None, 192)               1

**Tuning with Method 2: RandomSearch**

In [47]:
#OLD## Instantiate the RandomSearch tuner to perform the hypertuning with 20 random combinations

tuner1 = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=20,
    executions_per_trial=2,
    directory='My Drive',
    project_name='Colab Notebooks')


In [48]:
tuner1.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 218, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 218, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [49]:
tuner1.search(x_train, y_train,
             epochs=20,
             validation_split=0.3)

Trial 20 Complete [00h 00m 34s]
val_accuracy: 0.8083164393901825

Best val_accuracy So Far: 0.8160919547080994
Total elapsed time: 00h 09m 34s
INFO:tensorflow:Oracle triggered exit


In [52]:
# Get the optimal hyperparameters
best_hps=tuner1.get_best_hyperparameters(num_trials=1)[0]


In [53]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model1 = tuner1.hypermodel.build(best_hps)
history = model1.fit(x_train, y_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
124/124 [==============================] - 1s 4ms/step - loss: 12.6608 - accuracy: 0.6368 - val_loss: 0.8239 - val_accuracy: 0.7627
Epoch 2/50
124/124 [==============================] - 0s 3ms/step - loss: 1.3489 - accuracy: 0.7090 - val_loss: 0.9075 - val_accuracy: 0.8032
Epoch 3/50
124/124 [==============================] - 0s 3ms/step - loss: 1.3639 - accuracy: 0.7373 - val_loss: 0.5845 - val_accuracy: 0.6846
Epoch 4/50
124/124 [==============================] - 0s 3ms/step - loss: 1.4894 - accuracy: 0.7190 - val_loss: 0.4203 - val_accuracy: 0.8134
Epoch 5/50
124/124 [==============================] - 0s 3ms/step - loss: 0.8529 - accuracy: 0.7584 - val_loss: 1.1724 - val_accuracy: 0.7789
Epoch 6/50
124/124 [==============================] - 0s 2ms/step - loss: 1.2228 - accuracy: 0.7342 - val_loss: 1.2398 - val_accuracy: 0.5051
Epoch 7/50
124/124 [==============================] - 0s 3ms/step - loss: 0.7794 - accuracy: 0.7552 - val_loss: 0.9000 - val_accuracy: 0.5690
Epoch

In [54]:
hypermodel1 = tuner1.hypermodel.build(best_hps)

# Retrain the model
hypermodel1.fit(x_test, y_test, epochs=best_epoch)

Epoch 1/22
67/67 [==============================] - 1s 2ms/step - loss: 29.9245 - accuracy: 0.6258
Epoch 2/22
67/67 [==============================] - 0s 2ms/step - loss: 7.0754 - accuracy: 0.6497
Epoch 3/22
67/67 [==============================] - 0s 2ms/step - loss: 8.5293 - accuracy: 0.7038
Epoch 4/22
67/67 [==============================] - 0s 3ms/step - loss: 20.4603 - accuracy: 0.6117
Epoch 5/22
67/67 [==============================] - 0s 3ms/step - loss: 2.4299 - accuracy: 0.7086
Epoch 6/22
67/67 [==============================] - 0s 3ms/step - loss: 1.5516 - accuracy: 0.7588
Epoch 7/22
67/67 [==============================] - 0s 2ms/step - loss: 2.2049 - accuracy: 0.7266
Epoch 8/22
67/67 [==============================] - 0s 2ms/step - loss: 2.1117 - accuracy: 0.7232
Epoch 9/22
67/67 [==============================] - 0s 2ms/step - loss: 1.2004 - accuracy: 0.7297
Epoch 10/22
67/67 [==============================] - 0s 2ms/step - loss: 2.2541 - accuracy: 0.7166
Epoch 11/22
67/67

In [55]:
eval_result = hypermodel1.evaluate(x_test, y_test)
print("[test loss, test accuracy]:", eval_result)

67/67 [==============================] - 0s 1ms/step - loss: 0.5342 - accuracy: 0.7918
[test loss, test accuracy]: [0.5341918468475342, 0.7917652726173401]


In [56]:
hypermodel1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 160)               7520      
_________________________________________________________________
dense_1 (Dense)              (None, 192)               30912     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                12352     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 50,849
Trainable params: 50,849
Non-trainable params: 0
_________________________________________________________________


In [62]:
hypermodel1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 160)               7520      
_________________________________________________________________
dense_1 (Dense)              (None, 192)               30912     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                12352     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 50,849
Trainable params: 50,849
Non-trainable params: 0
_________________________________________________________________


In [58]:
hypermodel1.optimizer.get_config()


{'amsgrad': False,
 'beta_1': 0.9,
 'beta_2': 0.999,
 'decay': 0.0,
 'epsilon': 1e-07,
 'learning_rate': 0.001,
 'name': 'Adam'}

In [63]:
# save best model

hypermodel1.save("/content/drive/My Drive/hypermodel1_best.h5")
